In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/fake-news-classification/train (2).csv
/kaggle/input/fake-news-classification/evaluation.csv
/kaggle/input/fake-news-classification/test (1).csv


In [2]:
import pandas as pd
import numpy as np
import nltk
import string
import gensim

In [3]:
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from gensim.models import Word2Vec
from sklearn.metrics import accuracy_score, classification_report

In [4]:
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /usr/share/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [5]:
eng_stopw = stopwords.words('english')
lemmatizer = WordNetLemmatizer()
punctuation = string.punctuation

In [6]:
dataset = pd.read_csv('/kaggle/input/fake-news-classification/train (2).csv', delimiter = ';')

In [7]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24353 entries, 0 to 24352
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  24353 non-null  int64 
 1   title       24353 non-null  object
 2   text        24353 non-null  object
 3   label       24353 non-null  int64 
dtypes: int64(2), object(2)
memory usage: 761.2+ KB


In [8]:
dataset.drop(columns=['Unnamed: 0'] , inplace=True)

In [9]:
dataset.isnull().sum()

title    0
text     0
label    0
dtype: int64

In [10]:
duplicates = dataset.duplicated()
print(duplicates.sum())

1


In [11]:
dataset_cleaned = dataset.drop_duplicates()

In [12]:
dataset_cleaned = dataset_cleaned.reset_index(drop=True)
print(dataset_cleaned.index)

RangeIndex(start=0, stop=24352, step=1)


In [13]:
def preprocessing_sentence(text):
  words = word_tokenize(text.lower())

  words = [w for w in words if w not in eng_stopw and w not in punctuation and w.isalpha()]
  words = [lemmatizer.lemmatize(word) for word in words]

  return words

In [14]:
clean_text = dataset_cleaned['text'].apply(preprocessing_sentence)
clean_title = dataset_cleaned['title'].apply(preprocessing_sentence)

In [15]:
X = pd.DataFrame({'title': clean_title, 'text': clean_text})
y = dataset_cleaned['label']

In [16]:
X

,title,text
0,"[palestinian, switch, christmas, light, bethle...","[ramallah, west, bank, reuters, palestinian, s..."
1,"[china, say, trump, call, taiwan, president, w...","[beijing, reuters, donald, trump, call, taiwan..."
2,"[fail, trump, organization, credit, score, mak...","[controversy, trump, personal, tax, return, co..."
3,"[zimbabwe, military, chief, china, trip, norma...","[beijing, reuters, trip, beijing, last, week, ..."
4,"[uncourageous, president, ever, receives, cour...","[never, uncourageous, person, white, house, ba..."
...,...,...
24347,"[mexico, senate, committee, ok, air, transport...","[mexico, city, reuters, key, committee, mexico..."
24348,"[breaking, hillary, clinton, state, department...","[toast, bigger, trouble, thought, much, whole,..."
24349,"[trump, break, stump, speech, admire, beautifu...","[kremlin, nato, created, agression, russia, to..."
24350,"[nfl, player, delivers, courageous, message, s...","[dallas, cowboy, star, wide, receiver, dez, br..."


In [17]:
X['combined_text'] = X['title'] + X['text']

In [18]:
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp)

In [19]:
X_train = X_train.reset_index(drop=True)
y_train = y_train.reset_index(drop=True)
X_val = X_val.reset_index(drop=True)
y_val = y_val.reset_index(drop=True)
X_test = X_test.reset_index(drop=True)
y_test = y_test.reset_index(drop=True)

In [20]:
X_train['combined_text'] = X_train['title'] + X_train['text']
X_val['combined_text'] = X_val['title'] + X_val['text']
X_test['combined_text'] = X_test['title'] + X_test['text']

In [21]:
sentence = X_train['combined_text'].tolist()
word2vec_model = Word2Vec(sentence, vector_size=100, window=5, min_count=3, sg=1)
print(len(word2vec_model.wv))  # Untuk menghitung ada berapa kata unik di model
print(word2vec_model.wv.index_to_key[:10])
print(word2vec_model.wv.most_similar('people'))

34555
['trump', 'said', 'state', 'would', 'president', 'people', 'republican', 'one', 'year', 'new']
[('dispossessed', 0.683393657207489), ('person', 0.6661546230316162), ('sodomite', 0.6628092527389526), ('folk', 0.6619443297386169), ('others', 0.6594569683074951), ('penniless', 0.657457172870636), ('helpless', 0.6544690132141113), ('predominately', 0.6503061652183533), ('unbearable', 0.648444652557373), ('destitute', 0.6453390121459961)]


In [22]:
word2vec_model.save("word2vec_model_final.bin")

In [23]:
def get_vector(words, model, vector_size=100):
    vectors = [model.wv[word] for word in words if word in model.wv] # Cek apakah kata2
    # ada di model, karena dalam pelatihan model, terdapat min_count = 3 sehingga kata yang
    # frekuensinya < 3 akan dibuang

    if len(vectors) == 0:
        return np.zeros(vector_size)
    return np.mean(vectors, axis=0)

X_train_vectors = np.array([get_vector(words, word2vec_model) for words in sentence])

In [24]:
from xgboost import XGBClassifier

xgb_model = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
xgb_model.fit(X_train_vectors, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=None,
              n_jobs=None, num_parallel_tree=None, random_state=None, ...)

In [25]:
test_data = X_test['combined_text'].tolist()

In [26]:
X_test_vectors = np.array([get_vector(words, word2vec_model) for words in test_data])

In [27]:
predictions = xgb_model.predict(X_test_vectors)

In [28]:
accuracy = accuracy_score(y_test, predictions)
print("Accuracy:", accuracy)

Accuracy: 0.9471667122912675


In [29]:
import joblib
import xgboost as xgb

In [30]:
model_save_path = "/kaggle/working/xgboost_hoax_classifier_final.joblib"


joblib.dump(xgb_model, model_save_path)

['/kaggle/working/xgboost_hoax_classifier_final.joblib']